## Day 18

### Part 1 

As you look out the window and notice a heavily-forested continent slowly appear over the horizon, you are interrupted by the child sitting next to you. They're curious if you could help them with their math homework.

Unfortunately, it seems like this "math" follows different rules than you remember.

The homework (your puzzle input) consists of a series of expressions that consist of addition (+), multiplication (*), and parentheses ((...)). Just like normal math, parentheses indicate that the expression inside must be evaluated before it can be used by the surrounding expression. Addition still finds the sum of the numbers on both sides of the operator, and multiplication still finds the product.

However, the rules of operator precedence have changed. Rather than evaluating multiplication before addition, the operators have the same precedence, and are evaluated left-to-right regardless of the order in which they appear.

For example, the steps to evaluate the expression 1 + 2 * 3 + 4 * 5 + 6 are as follows:

- 1 + 2 * 3 + 4 * 5 + 6
-   3   * 3 + 4 * 5 + 6
-       9   + 4 * 5 + 6
-          13   * 5 + 6
-              65   + 6
-                  71

Parentheses can override this order; for example, here is what happens if parentheses are added to form 1 + (2 * 3) + (4 * (5 + 6)):

- 1 + (2 * 3) + (4 * (5 + 6))
- 1 +    6    + (4 * (5 + 6))
-      7      + (4 * (5 + 6))
-      7      + (4 *   11   )
-      7      +     44
-             51

Here are a few more examples:

- 2 * 3 + (4 * 5) becomes 26.
- 5 + (8 * 3 + 9 + 3 * 4 * 3) becomes 437.
- 5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) becomes 12240.
- ((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 becomes 13632.

Before you can help with the homework, you need to understand it yourself. Evaluate the expression on each line of the homework; what is the sum of the resulting values?

In [ ]:
import pandas as pd
import numpy as np

In [413]:
# Open file
f = open("input_data/Day18.txt", "r")

sum = 0
for line in f:
    
    line = line.strip('\n')
    sum += process_str(line)
    
f.close()

print('Sum is:', sum)

Sum is: 11076907812171


In [408]:
def split_strings(st):
    '''takes a string with at least one operator and returns the portion inside outermost parentheses 
    and the remainder of the string'''
    
    if st[0] != '(':
        return st[0], st[2:]
    
    else:
        depth_paren = 1
        ind = 1
        found_r_paren = False
    
        while (depth_paren > 0):
            if st[ind] == '(':
                depth_paren += 1
            elif st[ind] == ')':
                depth_paren -= 1
            ind += 1
        return st[1:ind-1], st[ind+1:]
    

In [409]:
split_strings('(2 * 3) + (2 + 3)')

('2 * 3', '+ (2 + 3)')

In [410]:
def process_str(st):
    '''this is a recursive function that calls itself upon hitting a left parenthese; otherwise it operates
    on the next number'''

    total = 0
    op = '+'
    
#    print('inside ', st)
    while len(st)>1:
        
        if st[2] in ['*','+']:  # we have a number followed by an operator
            
            if op == '*':
                total *= int(st[0])
            else:
                total += int(st[0])
        
            op, st = st[2], st[4:]
            
        else:   # we have a left paren
            
            substr, remainder = split_strings(st)
            
            if op == '*':
                total *= process_str(substr)
            else:
                total += process_str(substr)
                
            if remainder != '':
                op = remainder[0]
                st = remainder[2:]
            else: # we've hit the end
                return total
        
    # we have one more number to process
#    print('here', st)
    if op == '*':
        return total * int(st[0])
    else:
        return total + int(st[0])

In [407]:
process_str('(8 * (6 * 8 + 3) * 9 * 9 * 8) * 2')

528768

### Part 2
You manage to answer the child's questions and they finish part 1 of their homework, but get stuck when they reach the next section: advanced math.

Now, addition and multiplication have different precedence levels, but they're not the ones you're familiar with. Instead, addition is evaluated before multiplication.

For example, the steps to evaluate the expression 1 + 2 * 3 + 4 * 5 + 6 are now as follows:

- 1 + 2 * 3 + 4 * 5 + 6
-   3   * 3 + 4 * 5 + 6
-   3   *   7   * 5 + 6
-   3   *   7   *  11
-     21       *  11
-         231

Here are the other examples from above:

- 1 + (2 * 3) + (4 * (5 + 6)) still becomes 51.
- 2 * 3 + (4 * 5) becomes 46.
- 5 + (8 * 3 + 9 + 3 * 4 * 3) becomes 1445.
- 5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) becomes 669060.
- ((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 becomes 23340.

What do you get if you add up the results of evaluating the homework problems using these new rules?


In [7]:
import numpy as np

In [89]:
def evaluate_str(st):
    '''this function takes a string with no paren and returns a single value,
    performing + before *'''

    total = 0
    num_ops = st.split()
    while '+' in num_ops:
        plus_pos = num_ops.index('+')

        # sum values around the plus sign
        total = int(num_ops[plus_pos - 1]) + int(num_ops[plus_pos + 1])

        # delete first values and plus sign and replace second value with sum
        num_ops.pop(plus_pos)
        num_ops.pop(plus_pos)
        #string_total = str(total)
        num_ops[plus_pos - 1] = str(total)

    while '*' in num_ops:
        times_pos = num_ops.index('*')

        # sum values around the plus sign
        total = int(num_ops[times_pos - 1]) * int(num_ops[times_pos + 1])

        # delete first values and plus sign and replace second value with sum
        num_ops.pop(times_pos)
        num_ops.pop(times_pos)
        num_ops[times_pos - 1] = str(total)

    return total

In [83]:
def split_string(st):
    '''takes a string with at least one left paren and returns a left, middle, and right string'''
    
    lpos = st.find('(')
    
    # Grab left half of string
    if st[0] == '(':
        left = ''
    
    else:
        left = st[0:lpos]
    st = st[lpos:]
#    print(st)
    
    # find the middle
    depth_paren = 1
    ind = 1
    found_r_paren = False
    
    while (depth_paren > 0):
        if st[ind] == '(':
            depth_paren += 1
        elif st[ind] == ')':
            depth_paren -= 1
        ind += 1
        
    middle, right = st[1:ind-1], st[ind:]
    
    return left, middle, right

In [78]:
split_string('5 * (2 + (3 * 8)) + 1')

('5 * ', '2 + (3 * 8)', ' + 1')

In [87]:
def process_str(st):
    '''this is a recursive function that loops through replacing an expression in parentheses with its value'''

    while st.find('(') > -1:
        
        left, middle, right = split_string(st)
        st = left + str(process_str(middle)) + right
    
    # evalute the remainder
    return evaluate_str(st)

In [88]:
process_str('5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))')

returning from evaluate st with total: 56
returning from evaluate st with total: 14868
returning from evaluate st with total: 669060


669060

In [90]:
# Open file
f = open("input_data/Day18.txt", "r")

sum = 0
for line in f:
    
    line = line.strip('\n')
    sum += process_str(line)
    
f.close()

print('Sum is:', sum)

Sum is: 283729053022731
